### Interacting with the SHEEP server

Assuming the sheep server is running, and the ```sheep_client.py``` module has the correct BASE_URL, we can configure and run SHEEP jobs using the commands in this notebook.

First lets just set up a directory path (we only need this so we can find the circuit file later on):

In [ ]:
import os
if "SHEEP_HOME" in os.environ.keys():
  SHEEP_HOME = os.environ["SHEEP_HOME"]
else:
  SHEEP_HOME = os.path.join(os.environ["HOME"],"SHEEP","frontend")
import sys
sys.path.append(SHEEP_HOME)

In [ ]:
from pysheep import sheep_client
         

In [ ]:
sheep_client.new_job()

List the available contexts:

In [ ]:
sheep_client.get_available_contexts()

Lets do a test with HElib_F2:

In [ ]:
sheep_client.set_context("HElib_F2")

List the available input types:

In [ ]:
sheep_client.get_available_input_types()

Let's use uint16_t:

In [ ]:
sheep_client.set_input_type("uint8_t")

Now let's give it a circuit.  There are plenty of simple circuits in the ```benchmark_inputs/``` directory.  We need to provide the full path to the circuit though so we'll use the SHEEP_HOME we defined at the start.

In [ ]:
r=sheep_client.set_circuit(os.path.join(SHEEP_HOME,"benchmark_inputs/low_level/circuits/circuit-ADD-1.sheep"))

We can see the names of the "inputs" that this circuit expects:

In [ ]:
sheep_client.get_inputs()

Now we can set values for those inputs.  The circuit we've chosen just adds the two inputs together, so lets do 55+44 (and hope we get the answer 99 at the end!)

In [ ]:
sheep_client.set_inputs({"input_0":55,"input_1":44})

We have set the context, the input_type, the circuit_file and the input values - we should be ready to go!  Let's print out the configuration, and then check that we are fully configured.

In [ ]:
sheep_client.get_config()

In [ ]:
sheep_client.is_configured()

OK, let's execute the test!

In [ ]:
sheep_client.run_job()

The SHEEP server just used our selected "context" (i.e. HE library) to encrypt our inputs, evaluate our circuit, and decrypt the outputs.  It also then evaluated the same circuit "in the clear" (i.e. with no encryption) so that we can check we got the right answer.   The result, including the output values, the processing times, and the clear-text check, are available as a json object.

In [ ]:
r=sheep_client.get_results()
from pprint import pprint
pprint(r)

So we can see that 
 * we got the answer "99" (which is 55+44, the sum of our two inputs) 
 * all the output values we got (just one in this case) matched the result from the "cleartext" context.
 * from the timings (which are in microseconds), the encryption and decryption were fast, but evaluating the circuit took some time.